Here we'll try to create a new datapoint for the model to predict on, from a CraigsList post.

In [91]:
import requests, json, os, pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

from scipy.special import inv_boxcox

import nltk, string, os
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Trawl for Prospective Deals

In [2]:
API_KEY = 'RobertBo-cinemaro-PRD-171ca9e35-af8bdcbb' # Enter your API Key/"App ID" Here. Mine was 40 chars long.

In [3]:
FIND_ADVANCED = "findItemsAdvanced" # This is the eBay API endpoint service we'll be querying.

In [4]:
ELEC_GUITARS = '33034'
USED = '3000'
AUCTION = "Auction"
AUCTIONWITHBIN = "AuctionWithBIN"

In [5]:
ITEM_FILTER_0 = f'itemFilter(0).name=Condition&itemFilter(0).value={USED}' # Only used guitars
ITEM_FILTER_1 = f'itemFilter(1).name=HideDuplicateItems&itemFilter(1).value=true' # No duplicate listings
ITEM_FILTER_2 = f'itemFilter(2).name=MinPrice&itemFilter(2).value=1' # Only items that sell for > this value
ITEM_FILTER_3 = f'itemFilter(3).name=MaxQuantity&itemFilter(3).value=1' # No lots or batch sales. One item at a time
ITEM_FILTER_4 = f'itemFilter(4).name=MaxPrice&itemFilter(4).value=310' # Only items that sold for < this value

In [6]:
def find_current_auctions(PAGE, keywords):
    '''Make a request to the eBay API and return the JSON text of this page number'''
    r = requests.get(
                 f'https://svcs.ebay.com/services/search/FindingService/v1?'
                 f'OPERATION-NAME={FIND_ADVANCED}&'
                 f'X-EBAY-SOA-SECURITY-APPNAME={API_KEY}&'
                 f'RESPONSE-DATA-FORMAT=JSON&'
                 f'REST-PAYLOAD&'
                 f'categoryId={ELEC_GUITARS}&'
                 f'descriptionSearch=true&'
                 f'{ITEM_FILTER_0}&' # USED
                 f'{ITEM_FILTER_1}&' # NO DUPES
                 f'{ITEM_FILTER_2}&' # MINPRICE
                 f'{ITEM_FILTER_3}&' # NO LOTS
                 f'{ITEM_FILTER_4}&' # MAX PRICE
                 f'keywords={keywords}&'
                 f'paginationInput.pageNumber={str(PAGE)}') # value to be looped through when collecting lotsa data
    if r.json()['findItemsAdvancedResponse'][0].get('searchResult'):
        return r.json()['findItemsAdvancedResponse'][0]['searchResult'][0]['item']
    else:
        return None

In [7]:
def get_specs(ITEM_ID):
    '''Return the specifics of a single eBay auction. String input.'''
    r = requests.get('http://open.api.ebay.com/shopping?'
                    f'callname=GetSingleItem&'
                    f'responseencoding=JSON&'
                    f'appid={API_KEY}&'
#                     f'siteid=0&' # USA Store
                    f'version=967&' # What is this?
                    f'ItemID={ITEM_ID}&'
                    f'IncludeSelector=Details,ItemSpecifics,TextDescription')
    try:
        return r.json()['Item']
    except KeyError:
        pass

In [8]:
def persist_page_to_json(PAGE):
    '''Saves a page of JSON responses to one json per axe'''
    for i in range(len(PAGE)):
        with open("trawler_listings/axe_%s.json" % (PAGE[i]['itemId'][0]), 'w') as f:  # writing JSON object
            json.dump(PAGE[i], f)

In [9]:
def persist_spec_to_json(spec):
    '''Writes one page of Axe Specs to one json'''
    try:
        with open("trawler_specs/axe_%s.json" % (spec['ItemID']), 'w') as f:  # writing JSON object
            json.dump(spec, f)
    except TypeError:
        pass
    pass

In [10]:
def trawl_for_guitars(start_page, stop_page, fetch_function, keywords):
    existing_files = [name.split('_')[1].split('.')[0] for name in os.listdir('trawler_specs/') if not name.startswith('.')] # Ignore .DS_Store
    
    j = 0
    k = 0
    
    '''Spams the eBay API for pages of AXE DATA'''
    
    for i in range(start_page+1, stop_page+1):
        page = fetch_function(i, keywords)
        if page:
            persist_page_to_json(page)
            for axe in page:
                k += 1
                if axe['itemId'][0] not in existing_files:
                    j += 1
                    print('Get')
                    persist_spec_to_json(get_specs(axe['itemId'][0]))
                else:
                    print('Skip')
    
    print(f'\nChecked {k} guitars')
    print(f'\nGot {j} new guitars')

### Trawl

In [104]:
trawl_for_guitars(0,100,find_current_auctions,'%28fender+-squier%2C+american%2C+gibson%29')

Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip


KeyboardInterrupt: 

***

## Load the Prospects and Make Price Estimates

In [12]:
from Axe_Object import Axe

In [13]:
filenames = [name for name in os.listdir('trawler_specs/') if not name.startswith('.')] # Ignores hidden files on mac

In [14]:
prospects = []
for filename in filenames:
    try:
        this_axe = Axe('trawler_listings', 'trawler_specs', filename)
        if "LOT OF" not in this_axe.title.upper() and this_axe.price > 110 and this_axe.price < 800:
            if this_axe.string_config and this_axe.string_config < 5:
                continue
            if this_axe.market != 'EBAY-US':
                continue
            if this_axe.year and this_axe.year > 2019:
                continue
            prospects.append(this_axe)
    except ValueError:
        pass

In [15]:
len(prospects)

662

Establishing some default eBay auction settings here:

This is how you should sell stuff on eBay to maximize guitar price, basically.

In [45]:
# Properties
auction_duration    = pd.Series(np.full(len(prospects),7*24), name = 'auction_duration')
shipping_charged    = pd.Series(np.full(len(prospects),0), name = 'shipping_charged')          
seller_country_US   = pd.Series([1 for i in range(len(prospects))], name = 'seller_country_US')
autopay             = pd.Series([False for guitar in range(len(prospects))], name = 'autopay')
returns             = pd.Series([False for guitar in range(len(prospects))], name = 'returns')
listing_type_FixedPrice = pd.Series([True for guitar in range(len(prospects))], name = 'listing_type_FixedPrice')
ship_type_Free      = pd.Series([1 for guitar in range(len(prospects))], name = 'ship_type_Free')
ship_expedite       = pd.Series([0 for guitar in range(len(prospects))], name = 'ship_expedite')
start_hour          = pd.Series([1 for guitar in range(len(prospects))], name = 'start_hour_(7.667, 11.5]')
end_hour            = pd.Series([1 for guitar in range(len(prospects))], name = 'end_hour_(7.667, 11.5]')
start_weekday_6     = pd.Series([6 for guitar in range(len(prospects))], name = 'start_weekday_6')
end_weekday_6       = pd.Series([6 for guitar in range(len(prospects))], name = 'end_weekday_6')
# returns_time      = pd.Series([0 for guitar in range(len(prospects))], name = 'returns_time')
num_pics            = pd.Series([12 for guitar in range(len(prospects))], name = 'num_pics')
best_offer_enabled  = pd.Series([True for guitar in range(len(prospects))], name = 'best_offer_enabled')
ship_handling_time_2= pd.Series([1 for guitar in range(len(prospects))], name = 'ship_handling_time_2')
seller_positive_percent = pd.Series([1 for guitar in range(len(prospects))], name = 'seller_positive_percent_(99.5, 111.0]')
brand = pd.Series([guitar.brand for guitar in prospects], name = "brand")
body_type           = pd.Series([guitar.body_type for guitar in prospects], name = "body_type")
color               = pd.Series([guitar.color for guitar in prospects], name = "color")
right_left_handed   = pd.Series([guitar.right_left_handed for guitar in prospects], name = "right_left_handed")
string_config       = pd.cut(pd.Series([guitar.string_config for guitar in prospects], name = "string_config"),
                       [0,5,6,11,20])
country_manufacture = pd.Series([guitar.country_manufacture for guitar in prospects], name = "country_manufacture")
model_year = pd.cut(pd.Series([guitar.year for guitar in prospects], name = "model_year"), [1700,1975,1990,1995,2000,2005,2007,2010,2011,2012,2013,2015])

In [46]:
X_dummies = pd.concat([brand, color, country_manufacture, right_left_handed, best_offer_enabled, shipping_charged, returns, autopay, seller_country_US, ship_handling_time_2, listing_type_FixedPrice, ship_expedite, ship_type_Free, num_pics, auction_duration, start_hour, end_hour, start_weekday_6, end_weekday_6, seller_positive_percent, model_year, body_type, string_config],
              axis = 1)

X = pd.get_dummies(X_dummies, drop_first=True)

In [47]:
infile = open('bonus_columns','rb')
bonus_columns = pickle.load(infile)
infile.close()

In [66]:
fillers = []
remove = []
for col in bonus_columns:
    if col not in X.columns:
#         print(f'YAYAYAYA: {col}')
        filler = pd.Series(np.full(len(prospects),0), name=col)
        fillers.append(filler)
for col in X.columns:
    if col not in bonus_columns:
#         print(f'NUNUNUNU: {col}')
        X.drop(col, axis=1, inplace=True)
        
fillers_df = pd.concat(fillers, axis=1)

In [67]:
X_nontext = pd.concat([X, fillers_df], axis=1)

In [68]:
infile = open('saved_scaler','rb')
scaler = pickle.load(infile)
infile.close()

In [69]:
X_nontext.shape

(662, 119)

In [70]:
X_nontext_scaled = pd.DataFrame(scaler.transform(X_nontext))

/anaconda3/envs/Bayes/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype bool, uint8, int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [71]:
stemmer = SnowballStemmer("english")

In [72]:
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``', ",", ".", ":", "'s", "--","’"]

In [73]:
def assemble_guitar_document(axe):
    document = axe.title + ' '
    if axe.year != None:
        document += (str(axe.year) + ' ')
    if axe.material != None:
        document += axe.material + ' '
    if axe.model != None:
        document += axe.model + ' ' 
    if axe.brand != None:
        document += axe.brand + ' '
    if axe.subtitle != None:
        document += axe.subtitle + ' '
    if axe.condition_description != None:
        document += axe.condition_description + ' '
    if axe.description != None:
        document += axe.description
    return document

In [74]:
def process_doc(doc):
    stopwords_removed = ''
    tokens = nltk.word_tokenize(doc)
    for i in range(len(tokens)):
        if tokens[i].lower() not in stopwords_list and tokens[i] not in string.punctuation:
            stopwords_removed += stemmer.stem(tokens[i]) + ' '
    return stopwords_removed

In [75]:
raw_corpus = [assemble_guitar_document(guitar).lower() for guitar in prospects]

In [76]:
processed_text = pd.Series(list(map(process_doc, raw_corpus)), name = 'text')

In [77]:
infile = open('saved_vectorizer','rb')
vectorizer = pickle.load(infile)
infile.close()

In [78]:
tfidf = vectorizer.transform(processed_text)
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names())

In [101]:
X_ready = pd.concat([X_nontext, tfidf_df], axis=1)

In [83]:
infile = open('lasso_model','rb')
lasso_model = pickle.load(infile)
infile.close()

In [86]:
y_preds = lasso_model.predict(X_ready)

In [92]:
bxcx_lam = .4
y_preds_inv = inv_boxcox(y_preds, bxcx_lam)

But now...which ones could yield a respectable margin?

In [134]:
predicted_df = pd.concat([pd.Series(y_preds_inv), pd.Series([guitar.initial_price for guitar in prospects]),
                          pd.Series([guitar.title for guitar in prospects]), 
                          pd.Series([guitar.url for guitar in prospects])], axis=1)
predicted_df.columns = ['Estimate', 'Orig_Price','Title','Link']

In [135]:
predicted_df_sorted = predicted_df.sort_values('Estimate', ascending=False)

In [136]:
predicted_df_sorted.iloc[:20,:]

,Estimate,Orig_Price,Title,Link
511,1377.205638,189.95,Genuine 1995 Fender Standard Stratocaster Body...,http://www.ebay.com/itm/Genuine-1995-Fender-St...
113,1307.465419,260.00,2006 Fender Deluxe Stratocaster HSS w / Case! ...,http://www.ebay.com/itm/2006-Fender-Deluxe-Str...
115,1050.432546,255.00,2017 Charvel Pro-Mod San Dimas Style 2 HH! Tra...,http://www.ebay.com/itm/2017-Charvel-Pro-Mod-S...
649,1021.090117,127.50,2017 Jackson RR24 Pro Series Rhoads Shredder G...,http://www.ebay.com/itm/2017-Jackson-RR24-Pro-...
626,979.052425,200.00,Fender American Squier Bullet Stratocaster SSS...,http://www.ebay.com/itm/Fender-American-Squier...
481,937.997921,300.00,Esp LTD EC-256 Snow White,http://www.ebay.com/itm/Esp-LTD-EC-256-Snow-Wh...
85,923.174023,260.00,2017 Fender Classic Player Jaguar Special HH! ...,http://www.ebay.com/itm/2017-Fender-Classic-Pl...
50,920.357719,129.98,Vintage electric guitar rare Guitar VOX Strat...,http://www.ebay.com/itm/Vintage-electric-guita...
424,900.704737,179.99,Genuine Gibson Custom ES-175 Hardshell Case Bl...,http://www.ebay.com/itm/Genuine-Gibson-Custom-...
71,893.645190,280.00,UPGRADED CUSTOM WIRED DEVLIN DV400 LES PAUL SU...,http://www.ebay.com/itm/UPGRADED-CUSTOM-WIRED-...
